# Run all cells in order

In [0]:
!git clone https://github.com/naturomics/capsule-model-forked-from-Sarasra

In [0]:
cd capsule-model-forked-from-Sarasra

In [0]:
!wget "https://storage.googleapis.com/capsule_toronto/mnist_data.tar.gz"

In [0]:
!wget "https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz"

In [0]:
!wget "https://storage.googleapis.com/capsule_toronto/mnist_checkpoints.tar.gz"

In [0]:
!tar xvzf mnist_data.tar.gz

In [0]:
!ls

In [0]:
!tar xvzf cifar-10-binary.tar.gz

In [0]:
!tar xvzf mnist_checkpoints.tar.gz

In [0]:
#uploading weights if necessary
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!tar -czf hintonResultsCifar.tar.gz model.ckpt-24000.data-00000-of-00001

Edit this with your hyperparameters

In [0]:
%%writefile experiment.py
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Framework for training and evaluating models."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time
import numpy as np
from tqdm import tqdm
import tensorflow as tf


from input_data.cifar10 import cifar10_input
from input_data.mnist import mnist_input_record
from input_data.norb import norb_input_record
from models import capsule_model
from models import conv_model

FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_string('data_dir', 'mnist_data', 'The data directory.')
tf.flags.DEFINE_integer('eval_size', 10000, 'Size of the test dataset.')
tf.flags.DEFINE_string('hparams_override', None,
                       'A string of form key=value,key=value to override the'
                       'hparams of this experiment.')
tf.flags.DEFINE_integer('max_steps', 24000, 'Number of steps to train.')
tf.flags.DEFINE_string('model', 'baseline',
                       'The model to use for the experiment.'
                       'capsule or baseline')
tf.flags.DEFINE_string('dataset', 'mnist',
                       'The dataset to use for the experiment.'
                       'mnist, norb, cifar10.')
tf.flags.DEFINE_integer('num_gpus', 1, 'Number of gpus to use.')
tf.flags.DEFINE_integer('num_targets', 1,
                        'Number of targets to detect (1 or 2).')
tf.flags.DEFINE_integer('num_trials', 1,
                        'Number of trials for ensemble evaluation.')
tf.flags.DEFINE_integer('save_step', 1500, 'How often to save checkpoints.')
tf.flags.DEFINE_string('summary_dir', 'tmp3',
                       'Main directory for the experiments.')
tf.flags.DEFINE_string('checkpoint', 'tmp/train/model.ckpt-24000',
                       'The model checkpoint for evaluation.')
#tf.flags.DEFINE_string('checkpoint', 'mnist_checkpoint/model.ckpt-1',
#                      'The model checkpoint for evaluation.')
tf.flags.DEFINE_bool('train', True, 'Either train the model or test the model.')
tf.flags.DEFINE_bool('validate', False, 'Run training/eval in validation mode.')



models = {
    'capsule': capsule_model.CapsuleModel,
    'baseline': conv_model.ConvModel,
}


def get_features(split, total_batch_size, num_gpus, data_dir, num_targets,
                 dataset, validate=False):
  """Reads the input data and distributes it over num_gpus GPUs.

  Each tower of data has 1/FLAGS.num_gpus of the total_batch_size.

  Args:
    split: 'train' or 'test', split of the data to read.
    total_batch_size: total number of data entries over all towers.
    num_gpus: Number of GPUs to distribute the data on.
    data_dir: Directory containing the input data.
    num_targets: Number of objects present in the image.
    dataset: The name of the dataset, either norb or mnist.
    validate: If set, subset training data into training and test.

  Returns:
    A list of batched feature dictionaries.

  Raises:
    ValueError: If dataset is not mnist or norb.
  """

  batch_size = total_batch_size // max(1, num_gpus)
  features = []
  for i in range(num_gpus):
    with tf.device('/gpu:%d' % i):
      if dataset == 'mnist':
        features.append(
            mnist_input_record.inputs(
                data_dir=data_dir,
                batch_size=batch_size,
                split=split,
                num_targets=num_targets,
                validate=validate,
            ))
      elif dataset == 'norb':
        features.append(
            norb_input_record.inputs(
                data_dir=data_dir, batch_size=batch_size, split=split,
            ))
      elif dataset == 'cifar10':
        data_dir = os.path.join(data_dir, 'cifar-10-batches-bin')
        features.append(
            cifar10_input.inputs(
                split=split, data_dir=data_dir, batch_size=batch_size))
      else:
        raise ValueError(
            'Unexpected dataset {!r}, must be mnist, norb, or cifar10.'.format(
                dataset))
  return features


def extract_step(path):
  """Returns the step from the file format name of Tensorflow checkpoints.

  Args:
    path: The checkpoint path returned by tf.train.get_checkpoint_state.
      The format is: {ckpnt_name}-{step}

  Returns:
    The last training step number of the checkpoint.
  """
  file_name = os.path.basename(path)
  return int(file_name.split('-')[-1])


def load_training(saver, session, load_dir):
  """Loads a saved model into current session or initializes the directory.

  If there is no functioning saved model or FLAGS.restart is set, cleans the
  load_dir directory. Otherwise, loads the latest saved checkpoint in load_dir
  to session.

  Args:
    saver: An instance of tf.train.saver to load the model in to the session.
    session: An instance of tf.Session with the built-in model graph.
    load_dir: The directory which is used to load the latest checkpoint.

  Returns:
    The latest saved step.
  """
  if tf.gfile.Exists(load_dir):
    ckpt = tf.train.get_checkpoint_state(load_dir)
    if ckpt and ckpt.model_checkpoint_path:
      saver.restore(session, ckpt.model_checkpoint_path)
      prev_step = extract_step(ckpt.model_checkpoint_path)
    else:
      tf.gfile.DeleteRecursively(load_dir)
      tf.gfile.MakeDirs(load_dir)
      prev_step = 0
  else:
    tf.gfile.MakeDirs(load_dir)
    prev_step = 0
  return prev_step


def train_experiment(session, result, writer, last_step, max_steps, saver,
                     summary_dir, save_step):
  """Runs training for up to max_steps and saves the model and summaries.

  Args:
    session: The loaded tf.session with the initialized model.
    result: The resultant operations of the model including train_op.
    writer: The summary writer file.
    last_step: The last trained step.
    max_steps: Maximum number of training iterations.
    saver: An instance of tf.train.saver to save the current model.
    summary_dir: The directory to save the model in it.
    save_step: How often to save the model ckpt.
  """
  step = 0
  for i in tqdm(range(last_step, max_steps)):
    step += 1
    summary, _ = session.run([result.summary, result.train_op])
    writer.add_summary(summary, i)
    if (i + 1) % save_step == 0:
      saver.save(
          session, os.path.join(summary_dir, 'model.ckpt'), global_step=i + 1)


def load_eval(saver, session, load_dir):
  """Loads the latest saved model to the given session.

  Args:
    saver: An instance of tf.train.saver to load the model in to the session.
    session: An instance of tf.Session with the built-in model graph.
    load_dir: The path to the latest checkpoint.

  Returns:
    The latest saved step.
  """
  saver.restore(session, load_dir)
  print('model loaded successfully')
  return extract_step(load_dir)


def eval_experiment(session, result, writer, last_step, max_steps, **kwargs):
  """Evaluates the current model on the test dataset once.

  Evaluates the loaded model on the test data set with batch sizes of 100.
  Aggregates the results and writes one summary point to the summary file.

  Args:
    session: The loaded tf.session with the trained model.
    result: The resultant operations of the model including evaluation metrics.
    writer: The summary writer file.
    last_step: The last trained step.
    max_steps: Maximum number of evaluation iterations.
    **kwargs: Arguments passed by run_experiment but not used in this function.
  """
  del kwargs

  total_correct = 0
  total_almost = 0
  for _ in tqdm(range(max_steps)):
    summary_i, correct, almost = session.run(
        [result.summary, result.correct, result.almost])
    total_correct += correct
    total_almost += almost
    print(total_correct)
  total_false = max_steps * 100 - total_correct
  total_almost_false = max_steps * 100 - total_almost
  summary = tf.Summary.FromString(summary_i)
  summary.value.add(tag='correct_prediction', simple_value=total_correct)
  summary.value.add(tag='wrong_prediction', simple_value=total_false)
  summary.value.add(
      tag='almost_wrong_prediction', simple_value=total_almost_false)
  print('Total wrong predictions: {}, wrong percent: {}%'.format(
      total_false, total_false / max_steps))
  tf.logging.info('Total wrong predictions: {}, wrong percent: {}%'.format(
      total_false, total_false / max_steps))
  writer.add_summary(summary, last_step)


def run_experiment(loader,
                   load_dir,
                   writer,
                   experiment,
                   result,
                   max_steps,
                   save_step=0):
  """Starts a session, loads the model and runs the given experiment on it.

  This is a general wrapper to load a saved model and run an experiment on it.
  An experiment can be a training experiment or an evaluation experiment.
  It starts session, threads and queues and closes them before returning.

  Args:
    loader: A function of prototype (saver, session, load_dir) to load a saved
      checkpoint in load_dir given a session and saver.
    load_dir: The directory to load the previously saved model from it and to
      save the current model in it.
    writer: A tf.summary.FileWriter to add summaries.
    experiment: The function of prototype (session, result, writer, last_step,
      max_steps, saver, load_dir, save_step) which will execute the experiment
      steps from result on the given session.
    result: The resultant final operations of the built model.
    max_steps: Maximum number of experiment iterations.
    save_step: How often the training model should be saved.
  """
  config = tf.ConfigProto(allow_soft_placement=True)
  config.gpu_options.allow_growth=True
  session = tf.Session(config=config)
  init_op = tf.group(tf.global_variables_initializer(),
                     tf.local_variables_initializer())
  session.run(init_op)
  saver = tf.train.Saver(max_to_keep=1000)
  last_step = loader(saver, session, load_dir)
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(sess=session, coord=coord)
  try:
    experiment(
        session=session,
        result=result,
        writer=writer,
        last_step=last_step,
        max_steps=max_steps,
        saver=saver,
        summary_dir=load_dir,
        save_step=save_step)
  except tf.errors.OutOfRangeError:
    tf.logging.info('Finished experiment.')
  finally:
    coord.request_stop()
  coord.join(threads)
  session.close()


def train(hparams, summary_dir, num_gpus, model_type, max_steps, save_step,
          data_dir, num_targets, dataset, validate):
  """Trains a model with batch sizes of 128 to FLAGS.max_steps steps.

  It will initialize the model with either previously saved model in the
  summary directory or start from scratch if FLAGS.restart is set or the
  directory is empty.
  The training is distributed on num_gpus GPUs. It writes a summary at every
  step and saves the model every 1500 iterations.

  Args:
    hparams: The hyper parameters to build the model graph.
    summary_dir: The directory to save model and write training summaries.
    num_gpus: Number of GPUs to use for reading data and computation.
    model_type: The model architecture category.
    max_steps: Maximum number of training iterations.
    save_step: How often the training model should be saved.
    data_dir: Directory containing the input data.
    num_targets: Number of objects present in the image.
    dataset: Name of the dataset for the experiments.
    validate: If set, use training-validation set for training.
  """
  summary_dir += '/train/'
  with tf.Graph().as_default():
    # Build model
    features = get_features('train', 128, num_gpus, data_dir, num_targets,
                            dataset, validate)
    model = models[model_type](hparams)
    result, _ = model.multi_gpu(features, num_gpus)
    # Print stats
    param_stats = tf.contrib.tfprof.model_analyzer.print_model_analysis(
        tf.get_default_graph(),
        tfprof_options=tf.contrib.tfprof.model_analyzer.
        TRAINABLE_VARS_PARAMS_STAT_OPTIONS)
    sys.stdout.write('total_params: %d\n' % param_stats.total_parameters)
    """
    total_parameters = 0
    for ariable in tf.trainable_variables():
      local_parameters=1
      shape = variable.get_shape()
      for i in shape:
        print(shape)
        local_parameters*=i.values
        total_parameters+=local_parameters
    print(total)
    """
    writer = tf.summary.FileWriter(summary_dir)
    run_experiment(load_training, summary_dir, writer, train_experiment, result,
                   max_steps, save_step)
    writer.close()


def find_checkpoint(load_dir, seen_step):
  """Finds the global step for the latest written checkpoint to the load_dir.

  Args:
    load_dir: The directory address to look for the training checkpoints.
    seen_step: Latest step which evaluation has been done on it.
  Returns:
    The latest new step in the load_dir and the file path of the latest model
    in load_dir. If no new file is found returns -1 and None.

  """
  ckpt = tf.train.get_checkpoint_state(load_dir)
  if ckpt and ckpt.model_checkpoint_path:
    global_step = extract_step(ckpt.model_checkpoint_path)
    if int(global_step) != seen_step:
      return int(global_step), ckpt.model_checkpoint_path
  return -1, None


def evaluate(hparams, summary_dir, num_gpus, model_type, eval_size, data_dir,
             num_targets, dataset, validate, checkpoint=None):
  """Continuously evaluates the latest trained model or a specific checkpoint.

  Regularly (every 2 min, maximum 6 hours) checks the training directory for
  the latest model. If it finds any new model, it outputs the total number of
  correct and wrong predictions for the test data set to the summary file.
  If a checkpoint is provided performs the evaluation only on the specific
  checkpoint.

  Args:
    hparams: The hyperparameters for building the model graph.
    summary_dir: The directory to load training model and write test summaries.
    num_gpus: Number of GPUs to use for reading data and computation.
    model_type: The model architecture category.
    eval_size: Total number of examples in the test dataset.
    data_dir: Directory containing the input data.
    num_targets: Number of objects present in the image.
    dataset: The name of the dataset for the experiment.
    validate: If set, use validation set for continuous evaluation.
    checkpoint: (optional) The checkpoint file name.
  """
  load_dir = summary_dir + '/train/'
  summary_dir += '/test/'
  with tf.Graph().as_default():
    features = get_features('test', 100, num_gpus, data_dir, num_targets,
                            dataset, validate)
    model = models[model_type](hparams)
    result, _ = model.multi_gpu(features, num_gpus)
    test_writer = tf.summary.FileWriter(summary_dir)
    seen_step = -1
    paused = 0
    while paused < 360:
      print('start evaluation, model defined')
      if checkpoint:
        step = extract_step(checkpoint)
        last_checkpoint = checkpoint
      else:
        step, last_checkpoint = find_checkpoint(load_dir, seen_step)
      if step == -1:
        time.sleep(60)
        paused += 1
      else:
        paused = 0
        seen_step = step
        run_experiment(load_eval, last_checkpoint, test_writer, eval_experiment,
                       result, eval_size // 100)
   #     run_experiment(load_eval, last_checkpoint, test_writer, eval_experiment,
   #                    result, eval_size)
        if checkpoint:
          break

    test_writer.close()


def get_placeholder_data(num_steps, batch_size, features, session):
  """Reads the features into a numpy array and replaces them with placeholders.

  Loads all the images and labels of the features queue in memory. Replaces
  the feature queue reader handle with placeholders to switch input method from
  queue to placeholders. Using placeholders gaurantees the order of datapoints
  to stay exactly the same during each epoch.

  Args:
    num_steps: The number of times to read from the features queue.
    batch_size: The number of datapoints at each step.
    features: The dictionary containing the data queues such as images.
    session: The session handle to use for running tensors.

  Returns:
    data: List of numpy arrays containing all the queued data in features.
    targets: List of all the labels in range [0...num_classes].
  """
  image_size = features['height']
  depth = features['depth']
  print("depth value :"+depth)
  num_classes = features['num_classes']
  data = []
  targets = []
  for i in range(num_steps):
    data.append(
        session.run({
            'recons_label': features['recons_label'],
            'labels': features['labels'],
            'images': features['images'],
            'recons_image': features['recons_image']
        }))
    targets.append(data[i]['recons_label'])
  image_shape = (batch_size, depth, image_size, image_size)
  features['images'] = tf.placeholder(tf.float32, shape=image_shape)
  features['labels'] = tf.placeholder(
      tf.float32, shape=(batch_size, num_classes))
  features['recons_image'] = tf.placeholder(tf.float32, shape=image_shape)
  features['recons_label'] = tf.placeholder(tf.int32, shape=(batch_size))
  return data, targets


def infer_ensemble_logits(features, model, checkpoints, session, num_steps,
                          data):
  """Extracts the logits for the whole dataset and all the trained models.

  Loads all the checkpoints. For each checkpoint stores the logits for the whole
  dataset.

  Args:
    features: The dictionary of the input handles.
    model: The model operation graph.
    checkpoints: The list of all checkpoint paths.
    session: The session handle to use for running tensors.
    num_steps: The number of steps to run the experiment.
    data: The num_steps list of loaded data to be fed to placeholders.

  Returns:
    logits: List of all the final layer logits for different checkpoints.
  """
  _, inferred = model.multi_gpu([features], 1)
  logits = []
  saver = tf.train.Saver()
  for checkpoint in checkpoints:
    saver.restore(session, checkpoint)
    for i in range(num_steps):
      logits.append(
          session.run(
              inferred[0].logits,
              feed_dict={
                  features['recons_label']: data[i]['recons_label'],
                  features['labels']: data[i]['labels'],
                  features['images']: data[i]['images'],
                  features['recons_image']: data[i]['recons_image']
              }))
  return logits


def evaluate_ensemble(hparams, model_type, eval_size, data_dir, num_targets,
                      dataset, checkpoint, num_trials):
  """Evaluates an ensemble of trained models.

  Loads a series of checkpoints and aggregates the output logit of them on the
  test data. Selects the class with maximum aggregated logit as the prediction.
  Prints the total number of wrong predictions.

  Args:
    hparams: The hyperparameters for building the model graph.
    model_type: The model architecture category.
    eval_size: Total number of examples in the test dataset.
    data_dir: Directory containing the input data.
    num_targets: Number of objects present in the image.
    dataset: The name of the dataset for the experiment.
    checkpoint: The file format of the checkpoints to be loaded.
    num_trials: Number of trained models to ensemble.
  """
  checkpoints = []
  for i in range(num_trials):
    file_name = checkpoint.format(i)
    if tf.train.checkpoint_exists(file_name):
      checkpoints.append(file_name)

  with tf.Graph().as_default():
    batch_size = 100
    features = get_features('test', batch_size, 1, data_dir, num_targets,
                            dataset)[0]
    model = models[model_type](hparams)

    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth=True
    session = tf.Session(config=config)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=session, coord=coord)
    num_steps = eval_size // batch_size
    data, targets = get_placeholder_data(num_steps, batch_size, features,
                                         session)
    logits = infer_ensemble_logits(features, model, checkpoints, session,
                                   num_steps, data)
    coord.request_stop()
    coord.join(threads)
    session.close()

    logits = np.reshape(logits, (num_trials, num_steps, batch_size, -1))
    logits = np.sum(logits, axis=0)
    predictions = np.argmax(logits, axis=2)
    total_wrong = np.sum(np.not_equal(predictions, targets))
    print('Total wrooong predictions: {}, wrong percent: {}%'.format(
        total_wrong, total_wrong / eval_size * 100))


def default_hparams():
  """Builds an HParam object with default hyperparameters."""
  return tf.contrib.training.HParams(
      decay_rate=0.96,
      decay_steps=2000,
      leaky=False,
      learning_rate=0.001,
      loss_type='margin',
      num_prime_capsules=32,
      padding='VALID',
      remake=True,
      routing=3,
      verbose=False,
  )


def main(_):
  hparams = default_hparams()
  if FLAGS.hparams_override:
    hparams.parse(FLAGS.hparams_override)

  if FLAGS.train:
    train(hparams, FLAGS.summary_dir, FLAGS.num_gpus, FLAGS.model,
          FLAGS.max_steps, FLAGS.save_step, FLAGS.data_dir, FLAGS.num_targets,
          FLAGS.dataset, FLAGS.validate)
  else:
    if FLAGS.num_trials == 1:
      evaluate(hparams, FLAGS.summary_dir, FLAGS.num_gpus, FLAGS.model,
               FLAGS.eval_size, FLAGS.data_dir, FLAGS.num_targets,
               FLAGS.dataset, FLAGS.validate, FLAGS.checkpoint)
    else:
      evaluate_ensemble(hparams, FLAGS.model, FLAGS.eval_size, FLAGS.data_dir,
                        FLAGS.num_targets, FLAGS.dataset, FLAGS.checkpoint,
                        FLAGS.num_trials)


if __name__ == '__main__':
  tf.app.run()

Overwriting experiment.py


In [0]:
!python experiment.py 

In [0]:
#@title ModifiedMnistLoader
from keras.datasets import mnist
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils import to_categorical
import numpy as np
import cv2
import random
def load_modified_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_trainModified=np.zeros((60000,28,28,3),dtype=int)
    for x in range(x_train.shape[0]):
      x_trainModified[x] = cv2.cvtColor(x_train[x],cv2.COLOR_GRAY2RGB)
    x_testModified=np.zeros((10000,28,28,3),dtype=int)
    for x in range(x_test.shape[0]):
      x_testModified[x] = cv2.cvtColor(x_test[x],cv2.COLOR_GRAY2RGB)

    !wget "https://images2.alphacoders.com/101/1011957.jpg"
    sourceImage = mpimg.imread('1011957.jpg')

    sourceWidth = sourceImage.shape[1]
    sourceHeight = sourceImage.shape[0]
    width = 28
    height = 28

    for x in range(60000):
     x1 = random.randint(0, sourceWidth-width-1)
     y1 = random.randint(0, sourceHeight-height-1)
     croppedImage= sourceImage[y1:y1+height,x1:x1+width,:]
     for row in range(x_trainModified[x].shape[0]):
        for pixel in range(x_trainModified[x].shape[1]):
         if all(i<140 for i in x_trainModified[x][row][pixel]):
           x_trainModified[x][row][pixel]=croppedImage[row][pixel]

    for x in range(10000):
      x1 = random.randint(0, sourceWidth-width-1)
      y1 = random.randint(0, sourceHeight-height-1)
      croppedImage= sourceImage[y1:y1+height,x1:x1+width,:]
      for row in range(x_testModified[x].shape[0]):
        for pixel in range(x_testModified[x].shape[1]):
          if all(i<140 for i in x_testModified[x][row][pixel]):
            x_testModified[x][row][pixel]=croppedImage[row][pixel]
    
    x_trainModified = x_trainModified.reshape(-1, 28, 28, 3).astype('float32') / 255.
    x_testModified = x_testModified.reshape(-1, 28, 28, 3).astype('float32') / 255.
  #  y_train = to_categorical(y_train.astype('float32'))
  #  y_test = to_categorical(y_test.astype('float32'))
    return (x_trainModified, y_train), (x_testModified, y_test)

Using TensorFlow backend.


## Creating TF records MNIST

In [0]:
import gzip
import os

import numpy
from six.moves import urllib
import tensorflow as tf

In [0]:
params = {}
params['download_data_location'] = 'mnist_data/'
params['tfrecord_location'] = 'mnist_data/'

In [0]:
directory = params['download_data_location']
(train_images, train_labels), (test_images, test_labels) = load_modified_mnist()  

11501568/11490434 [==============================] - 2s 0us/step
--2019-11-20 12:21:54--  https://images2.alphacoders.com/101/1011957.jpg
Resolving images2.alphacoders.com (images2.alphacoders.com)... 104.20.15.194, 104.20.14.194, 2606:4700:10::6814:fc2, ...
Connecting to images2.alphacoders.com (images2.alphacoders.com)|104.20.15.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4390417 (4.2M) [image/jpeg]
Saving to: ‘1011957.jpg’

1011957.jpg         100%[===================>]   4.19M  2.77MB/s    in 1.5s    

2019-11-20 12:21:57 (2.77 MB/s) - ‘1011957.jpg’ saved [4390417/4390417]



In [0]:
!ls

1011957.jpg		input_data		  mnist_data.tar.gz  testdata
cifar-10-binary.tar.gz	mnist_checkpoints.tar.gz  models
experiment.py		mnist_data		  README.md


In [0]:
tf.gfile.MakeDirs(directory)
name = "train_2shifted_mnist.tfrecords"
filename = os.path.join(params['tfrecord_location'], name)
tfrecord_writer = tf.python_io.TFRecordWriter(filename)

In [0]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


In [0]:
train_images.shape

(60000, 28, 28, 3)

In [0]:
num_examples = train_images.shape[0]
images = train_images
labels = train_labels

rows = images.shape[1]
cols = images.shape[2]
depth = images.shape[3]
  
for index in range(num_examples):
  # 1. Convert your data into tf.train.Feature
  image_raw = images[index].tostring()
  feature = {
    'height': _int64_feature(rows),
    'width': _int64_feature(cols),
    'depth': _int64_feature(depth),
    'label': _int64_feature(labels[index]),
    'image_raw': _bytes_feature(image_raw)
  }
  # 2. Create a tf.train.Features
  features = tf.train.Features(feature=feature)
  # 3. Createan example protocol
  example = tf.train.Example(features=features)
  # 4. Serialize the Example to string
  example_to_string = example.SerializeToString()
  # 5. Write to TFRecord
  tfrecord_writer.write(example_to_string)

In [0]:
cd ..

/content/capsule-model-forked-from-Sarasra


In [0]:
rm -r mnist_data

In [0]:
!ls

train_2shifted_mnist.tfrecords


In [0]:
cd mnist_data

/content/capsule-model-forked-from-Sarasra/mnist_data


In [0]:
file_format = '{}_{}shifted_mnist.tfrecords'
data_dir='mnist_data'
split = 'train'
shift = 2
filenames = [os.path.join(data_dir, file_format.format(split, shift))]
with tf.name_scope('input'):
  filename_queue = tf.train.string_input_producer(
      filenames, shuffle=(split == 'train'))

In [0]:
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

W1120 12:28:17.955873 139805621323648 deprecation.py:323] From <ipython-input-35-201f3492d706>:1: __init__ (from tensorflow.python.ops.io_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.


In [0]:
# Convert to Examples and write the result to TFRecord files.
convert_and_save_to(train_images, train_labels, 'train', params)
convert_and_save_to(test_images, test_labels, 'test', params)

NameError: ignored

In [0]:
features = tf.parse_single_example(
      serialized_example,
      features={
          'image_raw': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'depth': tf.FixedLenFeature([], tf.int64)
      })

## Files copied to Ipython notebooks

In [0]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Tests for conv_model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import tensorflow as tf



class ConvModelTest(tf.test.TestCase):

  def setUp(self):
    self.hparams = tf.contrib.training.HParams(
        learning_rate=0.001,
        decay_rate=0.96,
        decay_steps=1,
        padding='SAME',
        verbose=False,
        loss_type='softmax')

  def testIntegrity(self):
    """Checks a multi_gpu call on ConvModel builds the desired graph.

    With the correct inference graph, multi_gpu is able to call inference
    multiple times without any increase in number of trainable variables or a
    duplication error. Each tower should have 4 set of (weight, bias) variable.
    """
    with tf.Graph().as_default():
      test_model = conv_model.ConvModel(self.hparams)
      toy_image = np.reshape(np.arange(32 * 32), (1, 1, 32, 32))
      input_image = tf.constant(toy_image, dtype=tf.float32)
      features = {
          'height': 32,
          'depth': 1,
          'images': input_image,
          'labels': tf.one_hot([2], 10),
          'num_targets': 1,
          'num_classes': 10,
      }
      _, tower_output = test_model.multi_gpu([features, features, features], 3)
      trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
      self.assertEqual(len(trainable_vars), 8)
      _, classes = tower_output[0].logits.get_shape()
      self.assertEqual(10, classes.value)



tf.test.main()

Running tests under Python 2.7.15: /usr/bin/python2
W1120 12:17:06.898376 139805621323648 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:12: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



UnrecognizedFlagError: ignored

In [0]:
#@title conv_model.py
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Convolutional Model class.

Uses only convolutional and fully connected layers for the inference ops.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


class ConvModel(model.Model):
  """A baseline multi GPU Model without capsule layers.

  The inference graph includes ReLU convolution layers and fully connected
  layers. The last layer is linear and has 10 units.
  """

  def _add_convs(self, input_tensor, channels):
    """Adds the convolution layers.

    Adds a series of convolution layers with ReLU nonlinearity and pooling
    after each of them.

    Args:
      input_tensor: a 4D float tensor as the input to the first convolution.
      channels: A list of channel sizes for input_tensor and following
        convolution layers. Number of channels in input tensor should be
        equal to channels[0].
    Returns:
      A 4D tensor as the output of the last pooling layer.
    """
    for i in xrange(1, len(channels)):
      with tf.variable_scope('conv{}'.format(i)) as scope:
        kernel = variables.weight_variable(
            shape=[5, 5, channels[i - 1], channels[i]], stddev=5e-2,
            verbose=self._hparams.verbose
        )
        conv = tf.nn.conv2d(
            input_tensor,
            kernel, [1, 1, 1, 1],
            padding=self._hparams.padding,
            data_format='NCHW')
        biases = variables.bias_variable([channels[i]],
                                         verbose=self._hparams.verbose)
        pre_activation = tf.nn.bias_add(conv, biases, data_format='NCHW')
        relu = tf.nn.relu(pre_activation, name=scope.name)
        if self._hparams.verbose:
          tf.summary.histogram('activation', relu)
        input_tensor = tf.contrib.layers.max_pool2d(
            relu, kernel_size=2, stride=2, data_format='NCHW', padding='SAME')

    return input_tensor

  def inference(self, features):
    """Adds the inference graph ops.

    Builds the architecture of the neural net to drive logits from features.
    The inference graph includes a series of convolution and fully connected
    layers and outputs a [batch, 10] tensor as the logits.

    Args:
      features: Dictionary of batched feature tensors like images and labels.
    Returns:
      A model.Inferred named tuple of expected outputs of the model like
      'logits' and 'remakes' for the reconstructions (to be added).
    """
    image = features['images']
    image_dim = features['height']
    image_depth = features['depth']
    image_4d = tf.reshape(image, [-1, image_depth, image_dim, image_dim])
    conv = self._add_convs(image_4d, [image_depth, 512, 256])
    hidden1 = tf.contrib.layers.flatten(conv)

    with tf.variable_scope('fc1') as scope:
      dim = hidden1.get_shape()[1].value
      weights = variables.weight_variable(shape=[dim, 1024], stddev=0.1,
                                          verbose=self._hparams.verbose)
      biases = variables.bias_variable(shape=[1024],
                                       verbose=self._hparams.verbose)
      pre_activation = tf.matmul(hidden1, weights) + biases
      hidden2 = tf.nn.relu(pre_activation, name=scope.name)

    with tf.variable_scope('softmax_layer') as scope:
      weights = variables.weight_variable(
          shape=[1024, features['num_classes']], stddev=0.1,
          verbose=self._hparams.verbose
      )
      biases = variables.bias_variable(shape=[features['num_classes']],
                                       verbose=self._hparams.verbose)
      logits = tf.matmul(hidden2, weights) + biases

    return model.Inferred(logits, None)

NameError: ignored

In [0]:
#@title model.py
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Basic Model class which provides the optimization wrap around inference.

A general model class wrapps the loss and optimizer operations around the
inference graph. Therefore, it should define the learning rate, global step
and optimizer. The current version supports a multiple gpu scenario by
enforcing single gpu to select one gpu for calculations and reuse variables.

Different models will only have different inference graphs and they share the
training and evaluation ops. Therefore, we define the inference function
as an abstract method that each model should define specifically for itself.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import collections
import tensorflow as tf


TowerResult = collections.namedtuple('TowerResult', ('inferred', 'almost',
                                                     'correct', 'grads'))
JoinedResult = collections.namedtuple('JoinedResult', ('summary', 'train_op',
                                                       'correct', 'almost'))
Inferred = collections.namedtuple('Inferred',
                                  ('logits', 'remakes'))


class Model(object):
  """Base class for building a model and running inference on it."""

  __metaclass__ = abc.ABCMeta

  def __init__(self, hparams):
    """Initializes the model parameters.

    Args:
      hparams: The hyperparameters for the model as tf.contrib.training.HParams.
    """
    self._hparams = hparams
    with tf.device('/cpu:0'):
      self._global_step = tf.get_variable(
          'global_step', [],
          initializer=tf.constant_initializer(0),
          trainable=False)

      learning_rate = tf.train.exponential_decay(
          learning_rate=hparams.learning_rate,
          global_step=self._global_step,
          decay_steps=hparams.decay_steps,
          decay_rate=hparams.decay_rate)
      learning_rate = tf.maximum(learning_rate, 1e-6)

      self._optimizer = tf.train.AdamOptimizer(learning_rate)

  @abc.abstractmethod
  def inference(self, features):
    """Adds the inference graph ops.

    Builds the architecture of the neural net to derive logits from features.
    The inference graph defined here should involve trainable variables
    otherwise the optimizer will raise a ValueError.

    Args:
      features: Dictionary of batched feature tensors like images and labels.
    Returns:
      An Inferred named tuple for expected outputs of the model like 'logits'
      and 'remakes' for the reconstructions.
    """
    raise NotImplementedError('Not implemented')

  def _single_tower(self, tower_ind, feature):
    """Calculates the model gradient for one tower.

    Adds the inference and loss operations to the graph. Calculates the
    gradients based on the loss. Appends all the output values of this tower to
    their respective lists.

    Args:
      tower_ind: The index number for this tower. Each tower is named as
                  tower_{tower_ind} and resides on gpu:{tower_ind}.
      feature: Dictionary of batched features like images and labels.
    Returns:
      A namedtuple TowerResult containing the inferred values like logits and
      reconstructions, gradients and evaluation metrics.
    """
    with tf.device('/gpu:%d' % tower_ind):
      with tf.name_scope('tower_%d' % (tower_ind)) as scope:
        inferred = self.inference(feature)
        losses, correct, almost = layers.evaluate(
            logits=inferred.logits,
            labels=feature['labels'],
            num_targets=feature['num_targets'],
            scope=scope,
            loss_type=self._hparams.loss_type,)
        tf.get_variable_scope().reuse_variables()
        grads = self._optimizer.compute_gradients(losses)

    return TowerResult(inferred, almost, correct, grads)

  def _average_gradients(self, tower_grads):
    """Calculate the average gradient for each variable across all towers.

    Args:
      tower_grads: List of gradient lists for each tower. Each gradient list
        is a list of (gradient, variable) tuples for all variables.
    Returns:
      List of pairs of (gradient, variable) where the gradient has been
      averaged across all towers.
    """
    average_grads = []
    for grads_and_vars in zip(*tower_grads):
      grads = tf.stack([g for g, _ in grads_and_vars])
      grad = tf.reduce_mean(grads, 0)

      v = grads_and_vars[0][1]
      grad_and_var = (grad, v)
      average_grads.append(grad_and_var)
    return average_grads

  def _summarize_towers(self, almosts, corrects, tower_grads):
    """Aggregates the results and gradients over all towers.

    Args:
      almosts: The number of almost correct samples for each tower.
      corrects: The number of correct samples for each tower.
      tower_grads: The gradient list for each tower.

    Returns:
      A JoinedResult of evaluation results, the train op and the summary op.
    """

    grads = self._average_gradients(tower_grads)
    train_op = self._optimizer.apply_gradients(
        grads, global_step=self._global_step)
    summaries = tf.get_collection(tf.GraphKeys.SUMMARIES)
    summary = tf.summary.merge(summaries)
    stacked_corrects = tf.stack(corrects)
    stacked_almosts = tf.stack(almosts)
    summed_corrects = tf.reduce_sum(stacked_corrects, 0)
    summed_almosts = tf.reduce_sum(stacked_almosts, 0)
    return JoinedResult(summary, train_op, summed_corrects, summed_almosts)

  def multi_gpu(self, features, num_gpus):
    """Build the Graph and add the train ops on multiple GPUs.

    Divides the inference and gradient computation on multiple gpus.
    Then aggregates the gradients and return the resultant ops.

    Args:
      features: A list of dictionary of different features of input data.
                len(features) should be at least num_gpus.
      num_gpus: Number of gpus to be distributed on.
    Returns:
      A tuple of JoinedResult output Ops to be called in Session.run for
      training, evaluation or visualization, such as train_op and merged
      summary and a list of inferred outputs of each tower.
    """
    almosts = []
    corrects = []
    tower_grads = []
    inferred = []
    with tf.variable_scope(tf.get_variable_scope()):
      for i in xrange(num_gpus):
        tower_output = self._single_tower(i, features[i])
        inferred.append(tower_output.inferred)
        almosts.append(tower_output.almost)
        corrects.append(tower_output.correct)
        tower_grads.append(tower_output.grads)

    summarized_results = self._summarize_towers(almosts, corrects, tower_grads)
    return summarized_results, inferred

In [0]:
#@title layers.py
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Library for capsule layers.

This has the layer implementation for coincidence detection, routing and
capsule layers.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf



def _squash(input_tensor):
  """Applies norm nonlinearity (squash) to a capsule layer.

  Args:
    input_tensor: Input tensor. Shape is [batch, num_channels, num_atoms] for a
      fully connected capsule layer or
      [batch, num_channels, num_atoms, height, width] for a convolutional
      capsule layer.

  Returns:
    A tensor with same shape as input (rank 3) for output of this layer.
  """
  with tf.name_scope('norm_non_linearity'):
    norm = tf.norm(input_tensor, axis=2, keep_dims=True)
    norm_squared = norm * norm
    return (input_tensor / norm) * (norm_squared / (1 + norm_squared))


def _leaky_routing(logits, output_dim):
  """Adds extra dimmension to routing logits.

  This enables active capsules to be routed to the extra dim if they are not a
  good fit for any of the capsules in layer above.

  Args:
    logits: The original logits. shape is
      [input_capsule_num, output_capsule_num] if fully connected. Otherwise, it
      has two more dimmensions.
    output_dim: The number of units in the second dimmension of logits.

  Returns:
    Routing probabilities for each pair of capsules. Same shape as logits.
  """

  # leak is a zero matrix with same shape as logits except dim(2) = 1 because
  # of the reduce_sum.
  leak = tf.zeros_like(logits, optimize=True)
  leak = tf.reduce_sum(leak, axis=2, keep_dims=True)
  leaky_logits = tf.concat([leak, logits], axis=2)
  leaky_routing = tf.nn.softmax(leaky_logits, dim=2)
  return tf.split(leaky_routing, [1, output_dim], 2)[1]


def _update_routing(votes, biases, logit_shape, num_dims, input_dim, output_dim,
                    num_routing, leaky):
  """Sums over scaled votes and applies squash to compute the activations.

  Iteratively updates routing logits (scales) based on the similarity between
  the activation of this layer and the votes of the layer below.

  Args:
    votes: tensor, The transformed outputs of the layer below.
    biases: tensor, Bias variable.
    logit_shape: tensor, shape of the logit to be initialized.
    num_dims: scalar, number of dimmensions in votes. For fully connected
      capsule it is 4, for convolutional 6.
    input_dim: scalar, number of capsules in the input layer.
    output_dim: scalar, number of capsules in the output layer.
    num_routing: scalar, Number of routing iterations.
    leaky: boolean, if set use leaky routing.

  Returns:
    The activation tensor of the output layer after num_routing iterations.
  """
  votes_t_shape = [3, 0, 1, 2]
  for i in range(num_dims - 4):
    votes_t_shape += [i + 4]
  r_t_shape = [1, 2, 3, 0]
  for i in range(num_dims - 4):
    r_t_shape += [i + 4]
  votes_trans = tf.transpose(votes, votes_t_shape)

  def _body(i, logits, activations):
    """Routing while loop."""
    # route: [batch, input_dim, output_dim, ...]
    if leaky:
      route = _leaky_routing(logits, output_dim)
    else:
      route = tf.nn.softmax(logits, dim=2)
    preactivate_unrolled = route * votes_trans
    preact_trans = tf.transpose(preactivate_unrolled, r_t_shape)
    preactivate = tf.reduce_sum(preact_trans, axis=1) + biases
    activation = _squash(preactivate)
    activations = activations.write(i, activation)
    # distances: [batch, input_dim, output_dim]
    act_3d = tf.expand_dims(activation, 1)
    tile_shape = np.ones(num_dims, dtype=np.int32).tolist()
    tile_shape[1] = input_dim
    act_replicated = tf.tile(act_3d, tile_shape)
    distances = tf.reduce_sum(votes * act_replicated, axis=3)
    logits += distances
    return (i + 1, logits, activations)

  activations = tf.TensorArray(
      dtype=tf.float32, size=num_routing, clear_after_read=False)
  logits = tf.fill(logit_shape, 0.0)
  i = tf.constant(0, dtype=tf.int32)
  _, logits, activations = tf.while_loop(
      lambda i, logits, activations: i < num_routing,
      _body,
      loop_vars=[i, logits, activations],
      swap_memory=True)

  return activations.read(num_routing - 1)


def capsule(input_tensor,
            input_dim,
            output_dim,
            layer_name,
            input_atoms=8,
            output_atoms=8,
            **routing_args):
  """Builds a fully connected capsule layer.

  Given an input tensor of shape `[batch, input_dim, input_atoms]`, this op
  performs the following:

    1. For each input capsule, multiples it with the weight variable to get
      votes of shape `[batch, input_dim, output_dim, output_atoms]`.
    2. Scales the votes for each output capsule by iterative routing.
    3. Squashes the output of each capsule to have norm less than one.

  Each capsule of this layer has one weight tensor for each capsules of layer
  below. Therefore, this layer has the following number of trainable variables:
    w: [input_dim * num_in_atoms, output_dim * num_out_atoms]
    b: [output_dim * num_out_atoms]

  Args:
    input_tensor: tensor, activation output of the layer below.
    input_dim: scalar, number of capsules in the layer below.
    output_dim: scalar, number of capsules in this layer.
    layer_name: string, Name of this layer.
    input_atoms: scalar, number of units in each capsule of input layer.
    output_atoms: scalar, number of units in each capsule of output layer.
    **routing_args: dictionary {leaky, num_routing}, args for routing function.

  Returns:
    Tensor of activations for this layer of shape
      `[batch, output_dim, output_atoms]`.
  """
  with tf.variable_scope(layer_name):
    # weights variable will hold the state of the weights for the layer
    weights = variables.weight_variable(
        [input_dim, input_atoms, output_dim * output_atoms])
    biases = variables.bias_variable([output_dim, output_atoms])
    with tf.name_scope('Wx_plus_b'):
      # Depthwise matmul: [b, d, c] ** [d, c, o_c] = [b, d, o_c]
      # To do this: tile input, do element-wise multiplication and reduce
      # sum over input_atoms dimmension.
      input_tiled = tf.tile(
          tf.expand_dims(input_tensor, -1),
          [1, 1, 1, output_dim * output_atoms])
      votes = tf.reduce_sum(input_tiled * weights, axis=2)
      votes_reshaped = tf.reshape(votes,
                                  [-1, input_dim, output_dim, output_atoms])
    with tf.name_scope('routing'):
      input_shape = tf.shape(input_tensor)
      logit_shape = tf.stack([input_shape[0], input_dim, output_dim])
      activations = _update_routing(
          votes=votes_reshaped,
          biases=biases,
          logit_shape=logit_shape,
          num_dims=4,
          input_dim=input_dim,
          output_dim=output_dim,
          **routing_args)
    return activations


def _depthwise_conv3d(input_tensor,
                      kernel,
                      input_dim,
                      output_dim,
                      input_atoms=8,
                      output_atoms=8,
                      stride=2,
                      padding='SAME'):
  """Performs 2D convolution given a 5D input tensor.

  This layer given an input tensor of shape
  `[batch, input_dim, input_atoms, input_height, input_width]` squeezes the
  first two dimmensions to get a 4D tensor as the input of tf.nn.conv2d. Then
  splits the first dimmension and the last dimmension and returns the 6D
  convolution output.

  Args:
    input_tensor: tensor, of rank 5. Last two dimmensions representing height
      and width position grid.
    kernel: Tensor, convolutional kernel variables.
    input_dim: scalar, number of capsules in the layer below.
    output_dim: scalar, number of capsules in this layer.
    input_atoms: scalar, number of units in each capsule of input layer.
    output_atoms: scalar, number of units in each capsule of output layer.
    stride: scalar, stride of the convolutional kernel.
    padding: 'SAME' or 'VALID', padding mechanism for convolutional kernels.

  Returns:
    6D Tensor output of a 2D convolution with shape
      `[batch, input_dim, output_dim, output_atoms, out_height, out_width]`,
      the convolution output shape and the input shape.
      If padding is 'SAME', out_height = in_height and out_width = in_width.
      Otherwise, height and width is adjusted with same rules as 'VALID' in
      tf.nn.conv2d.
  """
  with tf.name_scope('conv'):
    input_shape = tf.shape(input_tensor)
    _, _, _, in_height, in_width = input_tensor.get_shape()
    # Reshape input_tensor to 4D by merging first two dimmensions.
    # tf.nn.conv2d only accepts 4D tensors.

    input_tensor_reshaped = tf.reshape(input_tensor, [
        input_shape[0] * input_dim, input_atoms, input_shape[3], input_shape[4]
    ])
    input_tensor_reshaped.set_shape((None, input_atoms, in_height.value,
                                     in_width.value))
    conv = tf.nn.conv2d(
        input_tensor_reshaped,
        kernel,
        [1, 1, stride, stride],
        padding=padding,
        data_format='NCHW')
    conv_shape = tf.shape(conv)
    _, _, conv_height, conv_width = conv.get_shape()
    # Reshape back to 6D by splitting first dimmension to batch and input_dim
    # and splitting second dimmension to output_dim and output_atoms.

    conv_reshaped = tf.reshape(conv, [
        input_shape[0], input_dim, output_dim, output_atoms, conv_shape[2],
        conv_shape[3]
    ])
    conv_reshaped.set_shape((None, input_dim, output_dim, output_atoms,
                             conv_height.value, conv_width.value))
    return conv_reshaped, conv_shape, input_shape


def conv_slim_capsule(input_tensor,
                      input_dim,
                      output_dim,
                      layer_name,
                      input_atoms=8,
                      output_atoms=8,
                      stride=2,
                      kernel_size=5,
                      padding='SAME',
                      **routing_args):
  """Builds a slim convolutional capsule layer.

  This layer performs 2D convolution given 5D input tensor of shape
  `[batch, input_dim, input_atoms, input_height, input_width]`. Then refines
  the votes with routing and applies Squash non linearity for each capsule.

  Each capsule in this layer is a convolutional unit and shares its kernel over
  the position grid and different capsules of layer below. Therefore, number
  of trainable variables in this layer is:

    kernel: [kernel_size, kernel_size, input_atoms, output_dim * output_atoms]
    bias: [output_dim, output_atoms]

  Output of a conv2d layer is a single capsule with channel number of atoms.
  Therefore conv_slim_capsule is suitable to be added on top of a conv2d layer
  with num_routing=1, input_dim=1 and input_atoms=conv_channels.

  Args:
    input_tensor: tensor, of rank 5. Last two dimmensions representing height
      and width position grid.
    input_dim: scalar, number of capsules in the layer below.
    output_dim: scalar, number of capsules in this layer.
    layer_name: string, Name of this layer.
    input_atoms: scalar, number of units in each capsule of input layer.
    output_atoms: scalar, number of units in each capsule of output layer.
    stride: scalar, stride of the convolutional kernel.
    kernel_size: scalar, convolutional kernels are [kernel_size, kernel_size].
    padding: 'SAME' or 'VALID', padding mechanism for convolutional kernels.
    **routing_args: dictionary {leaky, num_routing}, args to be passed to the
      update_routing function.

  Returns:
    Tensor of activations for this layer of shape
      `[batch, output_dim, output_atoms, out_height, out_width]`. If padding is
      'SAME', out_height = in_height and out_width = in_width. Otherwise, height
      and width is adjusted with same rules as 'VALID' in tf.nn.conv2d.
  """
  with tf.variable_scope(layer_name):
    kernel = variables.weight_variable(shape=[
        kernel_size, kernel_size, input_atoms, output_dim * output_atoms
    ])
    biases = variables.bias_variable([output_dim, output_atoms, 1, 1])
    votes, votes_shape, input_shape = _depthwise_conv3d(
        input_tensor, kernel, input_dim, output_dim, input_atoms, output_atoms,
        stride, padding)

    with tf.name_scope('routing'):
      logit_shape = tf.stack([
          input_shape[0], input_dim, output_dim, votes_shape[2], votes_shape[3]
      ])
      biases_replicated = tf.tile(biases,
                                  [1, 1, votes_shape[2], votes_shape[3]])
      activations = _update_routing(
          votes=votes,
          biases=biases_replicated,
          logit_shape=logit_shape,
          num_dims=6,
          input_dim=input_dim,
          output_dim=output_dim,
          **routing_args)
    return activations


def _margin_loss(labels, raw_logits, margin=0.4, downweight=0.5):
  """Penalizes deviations from margin for each logit.

  Each wrong logit costs its distance to margin. For negative logits margin is
  0.1 and for positives it is 0.9. First subtract 0.5 from all logits. Now
  margin is 0.4 from each side.

  Args:
    labels: tensor, one hot encoding of ground truth.
    raw_logits: tensor, model predictions in range [0, 1]
    margin: scalar, the margin after subtracting 0.5 from raw_logits.
    downweight: scalar, the factor for negative cost.

  Returns:
    A tensor with cost for each data point of shape [batch_size].
  """
  logits = raw_logits - 0.5
  positive_cost = labels * tf.cast(tf.less(logits, margin),
                                   tf.float32) * tf.pow(logits - margin, 2)
  negative_cost = (1 - labels) * tf.cast(
      tf.greater(logits, -margin), tf.float32) * tf.pow(logits + margin, 2)
  return 0.5 * positive_cost + downweight * 0.5 * negative_cost


def evaluate(logits, labels, num_targets, scope, loss_type):
  """Calculates total loss and performance metrics like accuracy.

  Args:
    logits: tensor, output of the model.
    labels: tensor, ground truth of the data.
    num_targets: scalar, number of present objects in the image,
      i.e. the number of 1s in labels.
    scope: The scope to collect losses of.
    loss_type: 'sigmoid' (num_targets > 1), 'softmax' or 'margin' for margin
      loss.

  Returns:
    The total loss of the model, number of correct predictions and number of
    cases where at least one of the classes is correctly predicted.
  Raises:
    NotImplementedError: if the loss_type is not softmax or margin loss.
  """
  with tf.name_scope('loss'):
    if loss_type == 'sigmoid':
      classification_loss = tf.nn.sigmoid_cross_entropy_with_logits(
          labels=labels / 2.0, logits=logits)
    elif loss_type == 'softmax':
      classification_loss = tf.nn.softmax_cross_entropy_with_logits(
          labels=labels, logits=logits)
    elif loss_type == 'margin':
      classification_loss = _margin_loss(labels=labels, raw_logits=logits)
    else:
      raise NotImplementedError('Not implemented')

    with tf.name_scope('total'):
      batch_classification_loss = tf.reduce_mean(classification_loss)
      tf.add_to_collection('losses', batch_classification_loss)
  tf.summary.scalar('batch_classification_cost', batch_classification_loss)

  all_losses = tf.get_collection('losses', scope)
  total_loss = tf.add_n(all_losses, name='total_loss')
  tf.summary.scalar('total_loss', total_loss)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      _, targets = tf.nn.top_k(labels, k=num_targets)
      _, predictions = tf.nn.top_k(logits, k=num_targets)
      missed_targets = tf.contrib.metrics.set_difference(targets, predictions)
      num_missed_targets = tf.contrib.metrics.set_size(missed_targets)
      correct = tf.equal(num_missed_targets, 0)
      almost_correct = tf.less(num_missed_targets, num_targets)
      correct_sum = tf.reduce_sum(tf.cast(correct, tf.float32))
      almost_correct_sum = tf.reduce_sum(tf.cast(almost_correct, tf.float32))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
  tf.summary.scalar('accuracy', accuracy)
  tf.summary.scalar('correct_prediction_batch', correct_sum)
  tf.summary.scalar('almost_correct_batch', almost_correct_sum)
  return total_loss, correct_sum, almost_correct_sum


def reconstruction(capsule_mask, num_atoms, capsule_embedding, layer_sizes,
                   num_pixels, reuse, image, balance_factor):
  """Adds the reconstruction loss and calculates the reconstructed image.

  Given the last capsule output layer as input of shape [batch, 10, num_atoms]
  add 3 fully connected layers on top of it.
  Feeds the masked output of the model to the reconstruction sub-network.
  Adds the difference with reconstruction image as reconstruction loss to the
  loss collection.

  Args:
    capsule_mask: tensor, for each data in the batch it has the one hot
      encoding of the target id.
    num_atoms: scalar, number of atoms in the given capsule_embedding.
    capsule_embedding: tensor, output of the last capsule layer.
    layer_sizes: (scalar, scalar), size of the first and second layer.
    num_pixels: scalar, number of pixels in the target image.
    reuse: if set reuse variables.
    image: The reconstruction target image.
    balance_factor: scalar, downweight the loss to be in valid range.

  Returns:
    The reconstruction images of shape [batch_size, num_pixels].
  """
  first_layer_size, second_layer_size = layer_sizes
  capsule_mask_3d = tf.expand_dims(capsule_mask, -1)
  atom_mask = tf.tile(capsule_mask_3d, [1, 1, num_atoms])
  filtered_embedding = capsule_embedding * atom_mask
  filtered_embedding_2d = tf.contrib.layers.flatten(filtered_embedding)
  reconstruction_2d = tf.contrib.layers.stack(
      inputs=filtered_embedding_2d,
      layer=tf.contrib.layers.fully_connected,
      stack_args=[(first_layer_size, tf.nn.relu),
                  (second_layer_size, tf.nn.relu),
                  (num_pixels, tf.sigmoid)],
      reuse=reuse,
      scope='recons',
      weights_initializer=tf.truncated_normal_initializer(
          stddev=0.1, dtype=tf.float32),
      biases_initializer=tf.constant_initializer(0.1))

  with tf.name_scope('loss'):
    image_2d = tf.contrib.layers.flatten(image)
    distance = tf.pow(reconstruction_2d - image_2d, 2)
    loss = tf.reduce_sum(distance, axis=-1)
    batch_loss = tf.reduce_mean(loss)
    balanced_loss = balance_factor * batch_loss
    tf.add_to_collection('losses', balanced_loss)
    tf.summary.scalar('reconstruction_error', balanced_loss)

  return reconstruction_2d

In [0]:
#@title variables.py
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Utility functions for declaring variables and adding summaries.

It adds all different scalars and histograms for each variable and provides
utility functions for weight and bias variables.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


def weight_variable(shape, stddev=0.1, verbose=False):
  """Creates a CPU variable with normal initialization. Adds summaries.

  Args:
    shape: list, the shape of the variable.
    stddev: scalar, standard deviation for the initilizer.
    verbose: if set add histograms.

  Returns:
    Weight variable tensor of shape=shape.
  """
  with tf.device('/cpu:0'):
    with tf.name_scope('weights'):
      weights = tf.get_variable(
          'weights',
          shape,
          initializer=tf.truncated_normal_initializer(
              stddev=stddev, dtype=tf.float32),
          dtype=tf.float32)
  variable_summaries(weights, verbose)
  return weights


def bias_variable(shape, verbose=False):
  """Creates a CPU variable with constant initialization. Adds summaries.

  Args:
    shape: list, the shape of the variable.
    verbose: if set add histograms.

  Returns:
    Bias variable tensor with shape=shape.
  """
  with tf.device('/cpu:0'):
    with tf.name_scope('biases'):
      biases = tf.get_variable(
          'biases',
          shape,
          initializer=tf.constant_initializer(0.1),
          dtype=tf.float32)
  variable_summaries(biases, verbose)
  return biases


def variable_summaries(var, verbose):
  """Attaches a lot of summaries to a Tensor (for TensorBoard visualization).

  Args:
    var: tensor, statistic summaries of this tensor is added.
    verbose: if set add histograms.
  """
  if verbose:
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)

      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)
  else:
    pass


def activation_summary(x, verbose):
  """Creates summaries for activations.

  Creates a summary that provides a histogram and sparsity of activations.

  Args:
    x: Tensor
    verbose: if set add histograms.
  """
  if verbose:
    tf.summary.histogram('activations', x)
    tf.summary.scalar('sparsity', tf.nn.zero_fraction(x))
  else:
    pass